In [2]:
import xarray as xr
import os
import numpy as np
import pandas as pd

In [3]:
res = 2.8125
local_dir = f"E:\\weatherbench\\{res}deg"

In [ ]:
def get_daily_value_for_year(feature_info, lat, lon, year):
    feature, df_fld = feature_info
    d = xr.open_dataset(os.path.join(local_dir, feature, f"{feature}_{res}deg", f"{feature}_{year}_{res}deg.nc"))
    one_slice = dict(lat=slice(lat, lat+1), lon=slice(lon, lon+1))
    data = d.loc[one_slice].to_dataframe()
    temps = data.query(f"lon=={lon} and lat=={lat}")[df_fld]
    # hourly temperature: each day covers 24 entries. Take the daily average over 24hrs
    one_year_daily=np.array([np.mean(temps[i:i+24]) for i in range(0, len(temps), 24)])
    return one_year_daily[:365].reshape((1, 365))

In [57]:
d=xr.open_dataset("E:\\weatherbench\\2.8125deg\\toa_incident_solar_radiation\\toa_incident_solar_radiation_2.8125deg\\toa_incident_solar_radiation_1979_2.8125deg.nc")

In [51]:
d=xr.open_dataset("E:\\weatherbench\\2.8125deg\\2m_temperature\\2m_temperature_2.8125deg\\2m_temperature_1979_2.8125deg.nc")

In [52]:
d

<xarray.Dataset>
Dimensions:  (lat: 64, lon: 128, time: 8760)
Coordinates:
  * lat      (lat) float64 -88.59 -85.78 -82.97 -80.16 ... 82.97 85.78 88.59
  * lon      (lon) float64 0.0 2.812 5.625 8.438 ... 348.8 351.6 354.4 357.2
  * time     (time) datetime64[ns] 1979-01-01 ... 1979-12-31T23:00:00
Data variables:
    t2m      (time, lat, lon) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2019-11-07 07:51:20 GMT by grib_to_netcdf-2.14.0: /opt/ecmw...

In [59]:
lon = 0.0
lat = 1.40625
#data = d.query(f"lon=={lon} and lat=={lat}")
one_slice = dict(lat=slice(lat, lat), lon=slice(lon, lon))
data = d.sel(lon=lon, lat=lat)['tisr']
data

<xarray.DataArray 'tisr' (time: 8753)>
array([ 495762.38, 1667101.  , 2727078.5 , ...,       0.  ,       0.  ,
             0.  ], dtype=float32)
Coordinates:
    lon      float64 0.0
    lat      float64 1.406
  * time     (time) datetime64[ns] 1979-01-01T07:00:00 ... 1979-12-31T23:00:00
Attributes:
    units:      J m**-2
    long_name:  TOA incident solar radiation

In [53]:
lats = [lt.item() for lt in d['lat']]

In [8]:
lons = [ln.item() for ln in d['lon']]
lons

[0.0,
 2.8125,
 5.625,
 8.4375,
 11.25,
 14.0625,
 16.875,
 19.6875,
 22.5,
 25.3125,
 28.125,
 30.9375,
 33.75,
 36.5625,
 39.375,
 42.1875,
 45.0,
 47.8125,
 50.625,
 53.4375,
 56.25,
 59.0625,
 61.875,
 64.6875,
 67.5,
 70.3125,
 73.125,
 75.9375,
 78.75,
 81.5625,
 84.375,
 87.1875,
 90.0,
 92.8125,
 95.625,
 98.4375,
 101.25,
 104.0625,
 106.875,
 109.6875,
 112.5,
 115.3125,
 118.125,
 120.9375,
 123.75,
 126.5625,
 129.375,
 132.1875,
 135.0,
 137.8125,
 140.625,
 143.4375,
 146.25,
 149.0625,
 151.875,
 154.6875,
 157.5,
 160.3125,
 163.125,
 165.9375,
 168.75,
 171.5625,
 174.375,
 177.1875,
 180.0,
 182.8125,
 185.625,
 188.4375,
 191.25,
 194.0625,
 196.875,
 199.6875,
 202.5,
 205.3125,
 208.125,
 210.9375,
 213.75,
 216.5625,
 219.375,
 222.1875,
 225.0,
 227.8125,
 230.625,
 233.4375,
 236.25,
 239.0625,
 241.875,
 244.6875,
 247.5,
 250.3125,
 253.125,
 255.9375,
 258.75,
 261.5625,
 264.375,
 267.1875,
 270.0,
 272.8125,
 275.625,
 278.4375,
 281.25,
 284.0625,
 286.875

In [ ]:
def linearize(x):
    linearized = np.array([])
    for chunk in x:
        linearized = np.concatenate((linearized, chunk), axis=0)
    return linearized

def detrend(feature, lat, lon, years, nombor):
    """
    Extract the mean feature value for each day in a year over the period between y1 and y2 inclusive and use
    it to do seasonal detrending.
    """
    y1, y2 = years
    data = None
    for y in range(y1, y2+1):
        more_data = get_daily_value_for_year(feature, lat, lon, y)
        if data is None:
            data = more_data
        else:
            data = np.concatenate((data, more_data), axis=0)
#    np.savetxt(f"daily_averages{nombor}.txt", data, delimiter=',')
    # means is an array where mean[0] is the feature value average over the years between y1 and y2 on the 
    # on the first day of a year
    means = np.array([np.array([data[i, j] for i in range(y2-y1+1)]).mean() for j in range(365)])
    detrended = data - means
    return detrended